### Bibliotecas

In [14]:
import geopandas as gpd
import cx_Oracle
import os

### Diretórios

In [11]:
# DEFINIÇÕES DE DIRETÓRIOS
BASE_DIR = os.getcwd()
DIR_PAI = os.path.dirname(os.path.dirname(BASE_DIR))
DIR_COLETA_DADOS = os.path.join(DIR_PAI, "Coleta de Dados")
DIR_CORD = os.path.join(DIR_COLETA_DADOS, "COORDENADAS_REGIOES")
DIR_MUNICIPIO_SP = os.path.join(DIR_CORD, "cidade_sp.geojson")
DIR_ZONAS_SP = os.path.join(DIR_CORD, "zonas_sp.geojson")
DIR_DISTRITOS_SP = os.path.join(DIR_CORD, "distrito_sp.geojson")

### Funções Auxiliares

In [ ]:
# Conectar ao banco de dados Oracle
conn = cx_Oracle.connect(user='seu_usuario', password='sua_senha', dsn='seu_dsn')
cursor = conn.cursor()

#### INSERT COORDENADAS - TBL_Municipio, TBL_Zona, TBL_Distrito 

In [12]:
# Carregar o arquivo GeoJSON usando GeoPandas
gdf = gpd.read_file(DIR_MUNICIPIO_SP)

In [13]:
gdf

,CD_MUN,NM_MUN,AREA_KM2,CORD_CENTRAL,geometry
0,3550308,São Paulo,1521.11,POINT (-46.647972 -23.649892),"POLYGON ((-46.55066 -23.35714, -46.55011 -23.3..."


In [ ]:
# Iterar sobre os dados e inserir no banco de dados
for _, row in gdf.iterrows():
    nm_mun = row['nome_municipio']  # Supondo que esta seja a coluna com o nome do município
    sg_estado = row['sigla_estado']  # Supondo que esta seja a coluna com a sigla do estado
    area_km2 = row['area_km2']  # Supondo que esta seja a coluna com a área em km²
    geometry = row['geometry'].wkt  # Converter a geometria para Well-Known Text (WKT)
    
    # Inserir no banco de dados
    cursor.execute("""
        INSERT INTO TBL_Municipio (NM_MUN, SG_Estado, AREA_KM2, geometry)
        VALUES (:nm_mun, :sg_estado, :area_km2, SDO_GEOMETRY(:geometry, 4326))
    """, nm_mun=nm_mun, sg_estado=sg_estado, area_km2=area_km2, geometry=geometry)

In [ ]:
# Confirmar as inserções e fechar a conexão
conn.commit()
cursor.close()
conn.close()